# 第一部分 --  张量
原文地址如下
https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

## 第一部分-张量-example 3-4 自动求导


## 自动求导
PyTorch：张量和自动求导
在以上示例中，我们必须手动实现神经网络的前向和后向传递。对于小型的两层网络，手动实施反向传递并不是什么大问题，但是对于大型的复杂网络而言，可以很快变得非常繁琐。

幸运的是，我们可以使用自动微分 来自动计算神经网络中的反向传递。PyTorch中的 autograd软件包完全提供了此功能。使用autograd时，网络的前向传递将定义一个 计算图；图中的节点为张量，边为从输入张量产生输出张量的函数。然后通过该图进行反向传播，可以轻松计算梯度。

这听起来很复杂，在实践中使用起来非常简单。每个张量代表计算图中的一个节点。Ifx是一个Tensor， x.requires_grad=True然后x.grad是另一个Tensor，它持有x相对于某个标量值的梯度。

在这里，我们使用PyTorch张量和autograd来实现我们的正弦波，并带有三阶多项式示例；现在我们不再需要手动通过网络实现反向传递：

In [ ]:
# -*- coding: utf-8 -*-
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# 设置设备为CPU 在CPU上运行 如果要在显卡上运行就改字符串为GPU
# device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
# 默认情况下生成张量作为输入和输出requires_grad=False，这代表我们不需要在反向传递过程中针对这些张量计算梯度。

x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
# 为权重创建随机的张量。对于三阶多项式，我们需要4个权重：y = a + b x + c x ^ 2 + d x ^ 3 
# 设置require_grad = True表示我们要使用在反向传递过程中维护这些张量。
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

#学习率
learning_rate = 1e-6

for t in range(2000):
    # Forward pass: compute predicted y using operations on Tensors.
    # 前向传递：使用张量上的运算来计算预测的y。
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    # 使用张量上的运算来计算和打印损失。 
    # 现在损失是形状的张量（1，）loss.it（）获取损失中保存的标量值。
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    # 使用自动求导计算向后传递。
    # 此调用将计算与require_grad = True的所有张量有关的损耗梯度。
    # 在此之后，调用a.grad，b.grad。 c.grad和d.grad将由张量保存分别相对于a，b，c，d的损耗梯度。
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # 使用梯度下降手动更新权重。包装在torch.no_grad（）中 
    # 因为权重具有require_grad = True，但是我们不需要在自动求导中跟踪它。
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        # 更新权重后手动将梯度归零
        # pytorch不同于tf 我们需要手动的设置梯度归零
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

## PyTorch：定义新的自动梯度函数
每个原始的自动求导运算符实际上都是在Tensor上运行的两个函数。的正向函数从输入张量计算输出张量。该向后功能接收输出张量的梯度相对于一些标量值，并计算输入张量的梯度相对于该相同标量值。

在PyTorch中，我们可以通过定义torch.autograd.Function和实现forward 和backward函数的子类来轻松定义自己的自动求导运算。然后，我们可以通过构造实例并像调用函数一样调用新的自动求导运算，并传递包含输入数据的张量。

在此示例中，我们将模型定义为y=a+bP3(c+dx) 代替 y=a+bx+cx2+dx3， 在哪里P3(x)=12(5x3−3x) 是三阶勒让德多项式。我们编写了自己的自定义autograd函数，用于向前和向后计算P3，并使用它来实现我们的模型：

In [ ]:
# -*- coding: utf-8 -*-
import torch
import math


class LegendrePolynomial3(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """
    """
    我们可以通过继承torch.autograd.Function来实现我们自定义的求导函数，并且通过在张量上操作来实现前向传播和反向传播
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        """
        在正向传播过程中我们接受一个包含输入的张量并且返回一个包含输出的张量
        ctx是可以使用的上下文对象存放信息以进行向后计算。
        您可以通过使用ctx.save_for_backward方法在向后传递缓存任意对象 。
        """
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        """
        在后向传递中，我们收到包含损失梯度的张量 关于输出，我们需要计算基于输入的损失梯度。
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")  # Uncomment this to run on GPU
# 设置在CPU上计算

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
# 创建输入和输出张量 
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For this example, we need
# 4 weights: y = a + b * P3(c + d * x), these weights need to be initialized
# not too far from the correct result to ensure convergence.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
# 为权重创建随机的张量
# 还是这个例子，我们需要4个权重：y = a + b * P3（c + d * x），这些权重需要初始化距离正确结果不要太远，以确保收敛。
# 设置require_grad = True表示在反向传播的过程中为张量计算梯度
a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    # To apply our Function, we use Function.apply method. We alias this as 'P3'.
    # 我们使用Function.apply方法。我们将其别名为“ P3”
    P3 = LegendrePolynomial3.apply

    # Forward pass: compute predicted y using operations; we compute
    # P3 using our custom autograd operation.
    # 前向通过：使用运算计算预测的y；
    # 我们通过使用我们的自定义自动求导操作计算 P3
    y_pred = a + b * P3(c + d * x)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    # 自动求导来反向传播
    loss.backward()

    # Update weights using gradient descent
    # 更新权值
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        # 手动梯度清零
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} * P3({c.item()} + {d.item()} x)')